# Problem 1 - Used Cars Price Analysis


In [1]:
import pandas as pd
import numpy as np

### Stage I - Ingestion

In [2]:
df = pd.read_csv('../data_raw/train.csv')
print("Raw shape:", df.shape)
df.head()

Raw shape: (5847, 14)


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


### Stage II - Cleaning and Feature Engineering

In [3]:
# a) Missing values
print("Missing values:")
print(df.isnull().sum())

# Fill Seats with mode
df['Seats'].fillna(df['Seats'].mode()[0], inplace=True)

# Power, Engine, Mileage are critical for analysis so we drop rows with missing values
df.dropna(subset=['Power','Engine','Mileage'], inplace=True)

# We drop the whole column for New_Price (86% missing)
df.drop('New_Price', axis=1, inplace=True)

print("\nAfter dropping:", df.shape)

Missing values:
Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

After dropping: (5809, 13)


C:\Users\007\AppData\Local\Temp\ipykernel_4784\3221181988.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Seats'].fillna(df['Seats'].mode()[0], inplace=True)


In [4]:
# b) Remove units
df['Mileage'] = df['Mileage'].str.replace(' kmpl','').str.replace(' km/kg','').astype(float)
df['Engine'] = df['Engine'].str.replace(' CC','').astype(int)
df['Power'] = df['Power'].str.replace(' bhp','').str.replace('null','').astype(float)

# drop rows where Power is still NaN after cleaning
df.dropna(subset=['Power'], inplace=True)

print("Units removed")

Units removed


In [5]:
# c) One-hot encoding
df = pd.get_dummies(df, columns=['Fuel_Type','Transmission'], drop_first=True, dtype=int)
print("One-hot done\n")
df.head()

One-hot done



,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5.0,12.50,0,1
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199,88.70,5.0,4.50,1,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7.0,6.00,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5.0,17.74,0,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5.0,3.50,0,1


In [6]:
# d) Creating Car_Age as a new feature
df['Car_Age'] = 2025 - df['Year']
print("Car_Age created\n")
df.head()

Car_Age created



,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5.0,12.50,0,1,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199,88.70,5.0,4.50,1,1,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7.0,6.00,0,1,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5.0,17.74,0,0,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5.0,3.50,0,1,12


In [7]:
# Save clean data
df.to_csv('../data_clean/clean_data.csv', index=False)
print("Clean data saved, shape:", df.shape)

Clean data saved, shape: (5809, 14)


### Stage III - Analysis (Pandas Operations)

In [8]:
# 1. select
selected = df[['Name','Location','Year','Car_Age','Price','Mileage','Power']]

# 2. filter - young diesel cars
young_diesel = df[(df['Car_Age'] < 8) & (df['Fuel_Type_Petrol'] == 1)]

# 3. rename
df2 = df.rename(columns={'Kilometers_Driven':'KM_Driven', 'Price':'Price_Lakh'})

# 4. mutate - price per year
df2['Price_per_Year'] = df2['Price_Lakh'] / df2['Car_Age']

# 5. arrange - top 10 most expensive
top10 = df2.sort_values('Price_Lakh', ascending=False).head(10)

# 6. summarize + groupby
summary = df2.groupby('Location').agg(
    mean_price=('Price_Lakh','mean'),
    median_mileage=('Mileage','median'),
    avg_age=('Car_Age','mean'),
    count=('Name','count')
).round(2).sort_values('mean_price', ascending=False)

summary.to_markdown('../reports/summary.md')

print("Top 5 cities by avg price:")
summary.head()

Top 5 cities by avg price:


,mean_price,median_mileage,avg_age,count
Location,,,,
Coimbatore,15.14,18.16,9.55,629
Bangalore,13.61,17.00,12.10,348
Kochi,11.32,18.60,9.47,637
Hyderabad,10.00,19.00,12.17,709
Delhi,9.91,17.54,11.62,535


In [9]:
print("Top 10 most expensive cars:")
top10[['Name','Location','Year','Price_Lakh']]

Top 10 most expensive cars:


,Name,Location,Year,Price_Lakh
3952,Land Rover Range Rover 3.0 Diesel LWB Vogue,Hyderabad,2017,160.00
5620,Lamborghini Gallardo Coupe,Delhi,2011,120.00
5752,Jaguar F Type 5.0 V8 S,Hyderabad,2015,100.00
1457,Land Rover Range Rover Sport SE,Kochi,2019,97.07
1917,BMW 7 Series 740Li,Coimbatore,2018,93.67
1927,BMW 7 Series 740Li,Bangalore,2017,93.00
4556,Mercedes-Benz SLK-Class 55 AMG,Bangalore,2014,90.00
5380,BMW X6 xDrive 40d M Sport,Ahmedabad,2015,85.00
2035,Mercedes-Benz SLC 43 AMG,Coimbatore,2019,83.96
2347,Jaguar XJ 3.0L Portfolio,Delhi,2016,79.00
